In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import *
from deepgd.model import *

In [3]:
import torch
from torch import nn, jit

In [4]:
from attrs import define, NOTHING

In [5]:
model = Model(
    config=Model.Config(mean=False)
)
model = jit.script(model)
model = torch.compile(model)

In [6]:
model

OptimizedModule(
  (_orig_mod): RecursiveScriptModule(
    original_name=Model
    (linear): RecursiveScriptModule(original_name=Linear)
  )
)

In [ ]:
jit.save(model, 'model.pt')

In [5]:
generator = Generator(
    params=NOTHING,
    block_config=NOTHING,
    edge_net_config=NOTHING,
    gnn_config=NOTHING,
    edge_feat_expansion=NOTHING,
    eps=NOTHING
)
generator = jit.script(generator)
generator = torch.compile(generator)

/opt/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/torch/jit/annotations.py:386: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
  warnings.warn(
